In [6]:
import pandas as pd
import dask.dataframe as dd
from dask_ml import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [33]:
# Obtiene una lista con las columnas categoricas a partir de una diferencia de conjuntos
def getCategoricalColumns(df):
    cols = set(df.columns)
    numColumns = set(df._get_numeric_data().columns)
    
    return list(cols - numColumns)

# Convierte las variables categoricas en numericas en base a label encoding
def getNumericDataset(df):
    categoricalColumns = getCategoricalColumns(df)
    numDataset = df.drop(columns= categoricalColumns)
    categoricalDataset = df[categoricalColumns]

    le = preprocessing.LabelEncoder()
    for i in categoricalColumns:
        categoricalDataset[i] = le.fit_transform(categoricalDataset[i])
    
    for i in categoricalColumns:
        numDataset[i] = categoricalDataset[i]
    
    return numDataset

In [34]:
def scaler(df):
    scaler = StandardScaler()
    return scaler.fit_transform(df)

In [40]:
def reduce_csv(csv_file):
    df = dd.read_csv(csv_file)
    df = df.drop(columns = ["Unnamed: 0"])

    x = getNumericDataset(df)
    columns = list(x.columns)
    x = scaler(x)
    
    pca = PCA(n_components=.8, svd_solver='full')
    pca.fit(x)
    # 
    reduced = []
    for i in range(0, len(pca.explained_variance_ratio_)):
        reduced.append(columns[i])
    return reduced


In [41]:
reduce_csv("/home/alt9193/Documents/IA/DeepLearningBackend/examples/telecom_churn_me.csv")

['YEAR_JOINED',
 'CURRENT_YEAR',
 'BILL_AMOUNT',
 'PAID_AMOUNT',
 'PAYMENT_TRANSACTIONS',
 'PARTY_REV',
 'PREPAID_LINES',
 'POSTPAID_LINES',
 'OTHER_LINES',
 'LINE_REV',
 'MOUS_TO_LOCAL_MOBILES',
 'MOUS_FROM_LOCAL_MOBILES',
 'MOUS_TO_LOCAL_LANDLINES',
 'MOUS_FROM_LOCAL_LANDLINES',
 'MOUS_TO_INT_NUMBER']